Code for creating templates of regulations

In [1]:
import spacy

In [2]:
# test_text = input("Enter your testing text: ")
output_dir = '../../data/NER Model/flat_h_mixed'
nlp2 = spacy.load(output_dir)
# doc = nlp2(test_text)
# print(' ')
# for ent in doc.ents:
#     print(ent.start_char, ent.end_char,ent.label_)

In [3]:
import re
from nltk.tokenize import word_tokenize
from pprint import pprint
from collections import defaultdict 

In [4]:
bullet_regex = re.compile("^(\(?)([0-9]+|[a-z]+)((\)\.)|(\.)|\))")
first_index_regex = re.compile("^[0-9]+[A-Z]?.\ *?\(1\)")

def identifying_parts(rule,entities):
    e = []
    condition = []
    action = []
    e_cleaned = []
    
    Conditiontype1 = ['if', 'where', 'while', 'after', 'on', 'upon', 'in the case', 'in case']
    Conditiontype2 = ['whose', 'who', 'while', 'if', 'having', 'unless', 'that', 'required','once']
    
    rule = re.sub(first_index_regex,'',rule.strip())
    rule = re.sub(bullet_regex,"",rule)
    
    toked_rule = rule.lower().split(' ')
    start_token = toked_rule[0]
        
    all_ents = entities['cleanedUpEnts']
    parsed_ents = all_ents
    
    all_spans = []

    for ent in all_ents:
        match_obj = re.search(re.escape(ent),rule)
        span_obj = match_obj.span()
        all_spans.append(span_obj)
    
    none_flag = True
    if start_token in Conditiontype1 or  \
        toked_rule[0:3] == ['in','the','case'] or toked_rule[0:2] == ['in','case']:
        #Get closest entity:
        
        min_ele_index = all_spans.index(min(all_spans, key=lambda span_: span_[0]))

        e.append(all_ents[min_ele_index])
        e_cleaned.append(parsed_ents[min_ele_index])
        
        comma_index = rule.find(',')
        condition.append(rule[0:comma_index])

        shall_index = rule.find('shall')
        may_index = rule.find('may')
        
        shall_index = len(rule) if shall_index == -1 else shall_index
        may_index = len(rule) if may_index == -1 else may_index
        
        min_index  = min(shall_index,may_index)
        action.append(rule[min_index:])
        none_flag = False
    
    shall_index = rule.find('shall')
    may_index = rule.find('may')
    shall_index = len(rule) if shall_index == -1 else shall_index
    may_index = len(rule) if may_index == -1 else may_index
    min_index  = min(shall_index,may_index)
    
    if any([ele for ele in Conditiontype2 if ele in rule[:min_index]]):
        # Any before occurence shall
        all_condition_indices = [rule[:min_index].find(ele) for ele in Conditiontype2]
        if any([ele != -1 for ele in all_condition_indices]) and len(all_spans) > 0:            
            min_ele_index = all_spans.index(min(all_spans,key=lambda span_:span_[0]))
            
            condition_words_pos = [rule.find(ele) for ele in Conditiontype2]
            condition_words_pos = [len(rule) if ele == -1 else ele for ele in condition_words_pos]
            min_condition_word_pos = min(condition_words_pos)
            min_condition_word = Conditiontype2[condition_words_pos.index(min_condition_word_pos)] 
            all_possible_entities = [ele for ele in all_spans if ele[0] < min_condition_word_pos]

            if len(all_possible_entities) > 0:
                min_entity_index = all_spans.index(min(all_possible_entities,key = lambda span_:span_[0]))
                e.append(all_ents[min_ele_index])
                e_cleaned.append(parsed_ents[min_ele_index])
                shall_index = rule.find('shall')
                may_index = rule.find('may')

                shall_index = len(rule) if shall_index == -1 else shall_index
                may_index = len(rule) if may_index == -1 else may_index
                min_index  = min(shall_index,may_index)

                condition.append(rule[:min_index])
                action.append(rule[min_index:])
                none_flag = False
    
    comma_part = rule[min_index:]
    comma_flag = False
    if comma_part[:5] == "shall":
        if ',' in comma_part[5:8]:
            comma_flag = True
    elif comma_part[:3] == "may":
        if ',' in comma_part[3:5]:
            comma_flag = True
    if comma_flag:
        # immediately after a shall ","
        # 
        sub_rule = rule[min_index:]
        commas = [m.start() for m in re.finditer(',', rule[min_index:])]
        indices_list = [ele for ele in all_spans if ele[0] < min_index]
        if len(commas) >= 2 and len(indices_list) > 0:
            min_entity_index = all_spans.index(min(indices_list,key = lambda span_:span_[0]))
            
            e.append(all_ents[min_entity_index])
            e_cleaned.append(parsed_ents[min_entity_index])
            
            condition.append(sub_rule[commas[0]:commas[1]])
            action.append(sub_rule[commas[1]:])
            none_flag = False
            
    if none_flag:
        # Main 
        indices_list = [ele for ele in all_spans if ele[0] < min_index]
        if len(indices_list) > 0:
            min_entity_index = all_spans.index(min(indices_list,key = lambda span_:span_[0]))
            e.append(all_ents[min_entity_index])
            e_cleaned.append(parsed_ents[min_entity_index])
            
            condition.append("nil")
            shall_index = rule.find('shall')
            may_index = rule.find('may')

            shall_index = len(rule) if shall_index == -1 else shall_index
            may_index = len(rule) if may_index == -1 else may_index

            min_index  = min(shall_index,may_index)
            action.append(rule[min_index:])
    
    return e, condition, action, e_cleaned

In [5]:
def find_previous_rule(parsed_rules, rules, entity):
    parsed_rules.reverse()
    rules.reverse()
    for i in range(len(parsed_rules)):
        e,c,a,pe = parsed_rules[i]
        if entity in pe:
            return rules[i] 
    return None

In [6]:
def template(rules,entities):
    all_sets = []
    default = []
    dict_ = defaultdict(list)
    count_def = 0
    count_parsed = 0
    final_list = []
    
    for i in range(len(rules)):
        e = []
        c = []
        a = []
        # Property 
        pe = []
        
        entity = entities[str(i)]
        rule = rules[i]
        
        
        cleanedEnts = entity['cleanedUpEnts']
        pureEnts = cleanedEnts 
        
        if cleanedEnts == [] or (len(pureEnts) == 1 and 'Board' in pureEnts[0]):
            default.append(rule)
            final_list.append(rule)
            continue
#             print(rule)
            
        elif len(cleanedEnts) == 1:
            if 'such' in rule:
                ## Such || then next word is entity we are concerenced about
                ## action = r
                ## 
                e,c,a,pe = identifying_parts(rule,entity)
                if len(e) == 1:
                    prev_rule_for_context = find_previous_rule(all_sets,rules[:i],pe[0])
                    if prev_rule_for_context is not None:
                        a = [f"<context>{prev_rule_for_context}  - {e[0]} <context>" +" "+a[0]]
#                         a = [f"<context>{prev_rule_for_context} <context>"+ e[0] +" "+a[0]]
            else:
                e,c,a,pe = identifying_parts(rule,entity)
        else:
            if 'such' in rule or 'Such' in rule:
                lower_rule = rule
                lower_rule.lower()
                such_pos = lower_rule.find('such')
                all_spans = []
                all_ents = entity['cleanedUpEnts']
                for ent in all_ents:
                    match_obj = re.search(re.escape(ent),rule)
                    span_obj = match_obj.span()
                    all_spans.append(span_obj)
                                
                e,c,a,pe = identifying_parts(rule,entity)
                j = 0
                split_rules = rule.split(' ')
                
                while(j < len(e)):
                    if 'such' in split_rules[0] or 'Such' in split_rules[0]:
                        context_info = find_previous_rule(all_sets,rules[:i],pe[j])
                        if context_info :
#                             a[j] = context_info+e[j]+a[j]
                            a[j] = f"<context>{context_info} - {e[j]}<context> "+a[j]
                    else:
                        #Need to chnage it to pure entities
                        entity_considered = e[j]
                        if entity_considered in rules[0:such_pos] and entity_considered in rules[such_pos:]:
                            a[j] = f"<context> {rule} <context>" + a[j]
                        else:
                            context_info = find_previous_rule(all_sets,rules[:i],pe[j])
                            if context_info :
#                                 a[j] = context_info+e[j]+a[j]
                                a[j] = f"<context>{context_info} - {e[j]}<context> "+a[j]
                    j+=1
            else:
                e,c,a,pe = identifying_parts(rule,entity)
        
        #Identifying context
        reg_search = re.search(r'regulation [0-9]+',rule)
        if reg_search is not None:
#             print(reg_search.group(),e,a)
#             a = [f"<context>{reg_search.group()}<context> " + " "+ ent +" " + action for ent,action in zip(e,a)]
            a = [f"<context>{reg_search.group()} - {ent} <context> " + action for ent,action in zip(e,a)]
    
        subreg_search = re.search(r'sub-regulation \([0-9]+\)',rule)
        if subreg_search is not None:
#             a = [f"<context>{subreg_search.group()}<context> "+ ent+" " + action for ent,action in zip(e,a)]
            a = [f"<context>{subreg_search.group()} - {ent} <context> " + action for ent,action in zip(e,a)]
        all_sets.append([e,c,a,pe])
        
        j = 0
        if len(e) == 0:
            default.append(rule)
            final_list.append(rule)
        while(j < len(e)):
            if(c[j] != 'nil'):
                ent = pe[j]
                content = f"if [{c[j]}] : \n {a[j]}"
                dict_[ent].append((content,rule))
                final_list.append((ent,content))
            else:
                ent = pe[j]
                content = f"{a[j]}"
                dict_[ent].append((content,rule))
                final_list.append((ent,content))
            j+=1
    #Final Template
    return dict_,default,final_list

In [7]:
# import json

# with open('cleaned_insider_rules_for_templating.json','r') as f:
#     rules = json.load(f)

In [8]:
# len(rules)
# print(len(entities))

In [9]:
# import pickle
# ale = {}
# with open('tagged.obj','rb') as f:
#     out_entities = pickle.load(f)
# #     pprint(out_entities)
#     entities = {}
#     ale = set()
#     for i in range(len(out_entities)):
# #         print(out_entities[i])
#         mrule = rules[i]
#         for ele in out_entities[i]:
#             ale.add(ele['label'])
#         entities[str(i)] = {
#             "cleanedUpEnts": [mrule[ele['start']:ele['end']] for ele in out_entities[i] if ele['label'] not in ['time','Place','Object','Penalty','Legal Doc']]
#         }

In [10]:
# new_rules = []
# new_entities = {}
# counter = 0
# for i in range(len(rules)):
#     val  = rules[i]
#     v1 = re.match('^[0-9]+(substituted|inserted)',val.lower())
#     v2 = re.match('^Explanation',val)
#     v3 = re.match('^NOTE:',val)
#     if (v1 is not None) or (v2 is not None) or (v3 is not None):
#         pass
#     else:
#         new_rules.append(rules[i])
#         new_entities[str(counter)] = entities[str(i)]
#         counter+=1
# print(len(new_rules),len(new_entities))


In [11]:
def template_tagging(rules,entities):
    all_sets = []
    default = []
    dict_ = defaultdict(list)
    count_def = 0
    count_parsed = 0
    final_list = []
    ex = []
    for i in range(len(rules)):
#         print(i)
#         print(rules[i],entities[str(i)])
        e = []
        c = []
        a = []
        pe = []
        
        entity = entities[str(i)]
        rule = rules[i]
        
        cleanedEnts = entity['cleanedUpEnts']
        pureEnts = cleanedEnts
        ans = []

        if len(cleanedEnts) == 0:
            ex.append({"text": rule, "ents": ans, "title": None})
            continue
        
        e,c,a,pe = identifying_parts(rule,entity)
        
        
        for ent in e:
            match_obj = re.search(re.escape(ent),rule)
            if match_obj:
                span_obj =  match_obj.span()
                ans.append({"start":span_obj[0],"end":span_obj[1],"label":"ENT"})
        for ent in c:
            match_obj = re.search(re.escape(ent),rule)
            if match_obj:
                span_obj =  match_obj.span()
                ans.append({"start":span_obj[0],"end":span_obj[1],"label":"COND"})
        for ent in a:
            match_obj = re.search(re.escape(ent),rule)
            if match_obj:
                span_obj =  match_obj.span()
                ans.append({"start":span_obj[0],"end":span_obj[1],"label":"ACT"})
        
        ex.append({"text": rule, "ents": ans, "title": None})
        
    return ex

In [12]:
# ex = template_tagging(new_rules,new_entities)

In [13]:
import re

In [14]:
def template_pipeline(rules,ut_entities, file_name=""):
#     with open('/home/buggi/RA/data/parse_dumps/HTML_TEMPLATING_TEXT/1538990677517.json','r') as f:
#         rules = json.load(f)
    import pickle
    ale = {}
#     with open('/home/buggi/RA/data/parse_dumps/TEMPLATING NER/1538990677517_tagged.pkl','rb') as f:
#         ut_entities = pickle.load(f)
    out_entities = []
    for ele in rules:
        mini_entities = []
        for jele in ut_entities[ele]:
            s,e,i,x = jele
            mini_entities.append({"start":s,"end":e,"label":i})
        out_entities.append(mini_entities)
    entities = {}
    ale = set()
    for i in range(len(out_entities)):
        mrule = rules[i]
        for ele in out_entities[i]:
            ale.add(ele['label'])
        entities[str(i)] = {
            "cleanedUpEnts": [mrule[ele['start']:ele['end']] for ele in out_entities[i] if ele['label'] not in ['time','Place','Object','Penalty','Legal Doc']]
        }
    
    new_rules = []
    new_entities = {}
    new_indices = []
    counter = 0
    for i in range(len(rules)):
        val  = rules[i]
        v1 = re.match('^[0-9]+(substituted|inserted)',val.lower())
        v2 = re.match('^Explanation',val)
        v3 = re.match('^NOTE:',val)
        if (v1 is not None) or (v2 is not None) or (v3 is not None):
            pass
        else:
            new_rules.append(rules[i])
            new_entities[str(counter)] = entities[str(i)]
            counter+=1
#     print(len(new_rules),len(new_entities))
    a,b,c = template(new_rules,new_entities)
    
    import sys
    original_stdout = sys.stdout # Save a reference to the original standard output
    
    if file_name != "":
        with open(file_name, 'w') as f:
            sys.stdout = f 
            # Change the standard output to the file we created.
            
            for ele,kele in list(zip(c,new_rules)):
                cleaned_jele = "None - "
                if type(ele) is str:
                    print("Def :")
                    print(f"{cleaned_jele}",ele)
                    print("NR :",kele)
                if type(ele) is tuple:
                    print(ele[0] + " : ")
                    print(f"{cleaned_jele}",ele[1])
                    print("NR :",kele)
                print('--------------------------------------------------------')
            
            sys.stdout = original_stdout 
            # Reset the standard output to its original value

    

In [15]:
def template_tag_pipeline(rules, ut_entities):
#     with open('/home/buggi/RA/data/parse_dumps/HTML_TEMPLATING_TEXT/1538990677517.json','r') as f:
#     rules = json.load(f)
    import pickle
    ale = {}
#     with open('/home/buggi/RA/data/parse_dumps/TEMPLATING NER/1538990677517_tagged.pkl','rb') as f:
#     ut_entities = pickle.load(f)
    out_entities = []
    for ele in rules:
        mini_entities = []
        for jele in ut_entities[ele]:
            s,e,i,x = jele
            mini_entities.append({"start":s,"end":e,"label":i})
        out_entities.append(mini_entities)
    entities = {}
    ale = set()
    for i in range(len(out_entities)):
        mrule = rules[i]
        for ele in out_entities[i]:
            ale.add(ele['label'])
        entities[str(i)] = {
            "cleanedUpEnts": [mrule[ele['start']:ele['end']] for ele in out_entities[i] if ele['label'] not in ['time','Place','Object','Penalty','Legal Doc']]
        }
    
    new_rules = []
    new_entities = {}
    new_indices = []
    counter = 0
    for i in range(len(rules)):
        val  = rules[i]
        v1 = re.match('^[0-9]+(substituted|inserted)',val.lower())
        v2 = re.match('^Explanation',val)
        v3 = re.match('^NOTE:',val)
        if (v1 is not None) or (v2 is not None) or (v3 is not None):
            pass
        else:
            new_rules.append(rules[i])
            new_entities[str(counter)] = entities[str(i)]
            counter+=1
#     print(len(new_rules),len(new_entities))
    
    ex = template_tagging(new_rules,new_entities)
    return ex

In [16]:
import json
from glob import glob
from tqdm import tqdm
import os
# with open('../../data/ALL_REGULATIONS_JSON_FLATTENED_SPLIT_SUBREG/Substantial_Acquisition_of_Shares_and_Takeovers.json','r') as f:
#     datum = json.load(f)['Jul01_2020']

for file in tqdm(glob('../../data/ALL_REGULATIONS_JSON_FLATTENED_SPLIT_SUBREG/*.json')):
    folder_name = os.path.basename(file).split('.')[0]
    
#     print(file)
    with open(file,'r') as f:
        json_list = json.load(f)
    
    for key in json_list:
        datum = json_list[key]
        
        arr = []
        for cl in datum:
            arr += cl[2]

        for i,ele in enumerate(arr):
            arr[i] = re.sub(r"\([1-9][0-9]*\)\ *?","",ele,count=1)
        entities = {}
        for i,ele in enumerate(arr):
            doc = nlp2(ele)
            entities[ele] = []
            for ent in doc.ents:
                entities[ele].append((ent.start_char, ent.end_char,ent.label_,ent.text))
        try:        
            ex2 = template_tag_pipeline(arr, entities)
            file_name = folder_name + "_" + key
    #         print(file_name)

            with open(f'../../data/TEMPLATE_FILLING_ALL_DATA/Entities_Tagged/{file_name}.json','w') as f:
                json.dump(ex2,f)

            import spacy
            from spacy import displacy

            colors =  {"ENT": " #9b59b6","COND":"#138d75","ACT":" #a9cce3"}
            options = {"ents": ["ENT","COND","ACT"], "colors": colors}

            html = displacy.render(ex2, style="ent", manual=True,options=options,jupyter=False)
            with open(f'../../data/TEMPLATE_FILLING_ALL_DATA/Entities_HTML_View/{file_name}.html','w') as f:
                f.write(html)

            template_pipeline(arr,entities,file_name=f'../../data/TEMPLATE_FILLING_ALL_DATA/Entities_Text/{file_name}.txt')
       
        except:
            print("NOT WORKING: ",file_name)
        

 47%|████▋     | 22/47 [00:33<00:35,  1.42s/it]

NOT WORKING:  Credit_Rating_Agencies_May30_2018


100%|██████████| 47/47 [01:08<00:00,  1.45s/it]


In [ ]:
ex2 = template_tag_pipeline(arr, entities)

In [137]:
with open('../../tmp/testing.json','w') as f:
    json.dump(ex2,f)

In [138]:
import spacy
from spacy import displacy

colors =  {"ENT": " #9b59b6","COND":"#138d75","ACT":" #a9cce3"}
options = {"ents": ["ENT","COND","ACT"], "colors": colors}

html = displacy.render(ex2, style="ent", manual=True,options=options,jupyter=False)
with open('sast_entities_new.html','w') as f:
    f.write(html)

In [139]:
# len(new_indices)
%ls /home/buggi/RA/data/parse_dumps/HTML_TEMPLATING_TEXT/1538990677517.json

/home/buggi/RA/data/parse_dumps/HTML_TEMPLATING_TEXT/1538990677517.json


In [26]:
template_pipeline(arr,entities)

98 98
Def :
None -    No acquirer shall acquire shares or voting rights in a target company which takentogether with shares or voting rights, if any, held by him and by persons acting inconcert with him in such target company, entitle them to exercise twenty-five percent  or  more  of  the  voting  rights  in  such  target  company  unless  the  acquirermakes  a  public  announcement  of  an  open  offer  for  acquiring  shares  of  suchtarget company in accordance with these regulations.2016, w.e.f. 25-05-2016.10 Clause  (ze)  renumbered  as  clause  “(zf)”  by  the  SEBI  (Substantial  Acquisition  of  Shares  and  Takeovers)(Second Amendment) Regulations, 2016, w.e.f. 25-05-2016.11  Substituted  by  the  SEBI  (Substantial  Acquisition  of  Shares  and  Takeovers)  (Second  Amendment)Regulations, 2018, w.e.f. 11-09-2018. Prior to the amendment, it read as “Companies Act, 1956 (1 of 1956)”.(2)  No acquirer, who together with persons acting in concert with him, has acquiredand holds i

In [45]:
# print(len(a),len(b),len(c))

NameError: name 'a' is not defined

In [35]:
for ele,kele in list(zip(c,new_rules)):
    cleaned_jele = ""
#     "None -" 
#     if jele is None else (f"{jele[0]}.({jele[1]}) -")
    if type(ele) is str:
#         print("Def :")
        print(f"{cleaned_jele}",ele)
#         print("NR :",kele)
    if type(ele) is tuple:
        print(ele[0] + " : ")
        print(f"{cleaned_jele}",ele[1])
#         print("NR :",kele)
    print('--------------------------------------------------------')

 These regulations may be called the SEBI (Prohibition of Insider Trading) Regulations, 2015.
--------------------------------------------------------
 These regulations shall come into force on the one hundred and twentieth day from the date of its publication in the Official Gazette.
--------------------------------------------------------
 In these regulations, unless the context otherwise requires, the following words, expressions and derivations therefrom shall have the meanings assigned to them as under:–
(a)“Act” means the Securities and Exchange Board of India Act, 1992 (15 of 1992);
(b)“Board” means the Securities and Exchange Board of India;
(c)“compliance officer” means any senior officer, designated so and reporting to the board of directors or head of the organization in case board is not there, who is financially literate and is capable of appreciating requirements for legal and regulatory compliance under these regulations and who shall be responsible for compliance of p

In [ ]:
# for ele,val in a.items():
# #     for elem in val:
#     print(ele)
# #         print(elem[0])
# #         print(elem[1])
# #         print(',,,,,,,,')
#     print('---------------')

In [ ]:
# # print(type(a))
# sum_ = 0
# for ele,val in a.items():
#     sum_ += len(val)
#     print("Case :: "+ele)
#     for jele in val:
#         print("Converted Rule:",jele[0])
#         print("Original Rule:",jele[1])
#         print(',,,,,,')
#     print('----')
# sum_
# print("Default Rules ::")
# for ele in b:
#     print(ele)
#     print()

In [143]:
import csv
file = open('rule_templating_output_enfin.csv', 'w+', newline ='') 
# writing the data into the file 
with file:     
    write = csv.writer(file)
    write.writerows([[ele] for ele in b]) 

In [ ]:
import csv
with open('rule_templating_output.csv', 'r') as file:
    reader = csv.reader(file)
    old_b = [row for row in reader]

In [ ]:
print(len(old_b))

In [ ]:
import csv
with open('rule_templating_output_new.csv', 'r') as file:
    reader = csv.reader(file)
    b = [row for row in reader]

In [ ]:
ans_ = []
for ele in b:
    if ele not in old_b:
        ans_.append(ele)

In [ ]:
len(ans_)

In [61]:
# ls 'insider_rule_dump.json'
import json

In [53]:
with open('insider_rule_dump.json') as f:
    data = json.load(f)
import re
#The following functions are used to index the documents is order and combine them chapter wise.
p = re.compile(r'^(\(\d\)|\(\d\d\))')
new_data = []
int_list = []
for k in range(len(data)):
    int_list.append(data[k])
    if k+1<len(data):
        if not p.findall(data[k+1]):
            pass
        else:
            new_data.append(''.join(map(str, int_list)))
            int_list = []

for line in new_data:
    p = re.compile(r'^(\(\d\)|\(\d\d\))')

reg_data = []
chap_list = []
for k in range(len(new_data)):
    chap_list.append(new_data[k])
    if k+1<len(new_data):
        if p.findall(new_data[k+1])[0][1]=='1':
            reg_data.append(chap_list)
            chap_list = []
reg_dict = {}
#reg_dict is a dictionary of rules with keys as chapter numbers and values as list of ordered rules(strings).
count = 0
for k in reg_data:
    rules_dict = {}
    for i in k:
        rules_dict[p.findall(i)[0][1]] = i 
    count +=1
    reg_dict[count] = rules_dict

In [54]:
# print(int_list)

In [55]:
with open('insider_rule_dump.json') as f:
    data = json.load(f)
    print(len(data))
# import json
with open('cleaned_insider_rules_for_templating.json','r') as f:
    rules = json.load(f)
    print(len(rules))

123
123


In [56]:
final_dict = {} 
for ele in reg_dict:
    for jele in reg_dict[ele]:
        val_ = reg_dict[ele][jele]
        final_dict[val_] = [ele,jele]

with open('insider_rule_dump.json','r') as f:
    uncleaned_rules = json.load(f)
    
indices = []
for i in range(len(uncleaned_rules)): 
    if uncleaned_rules[i] in final_dict:
#         print(uncleaned_rules[i][:10],final_dict[uncleaned_rules[i]])
        indices.append(final_dict[uncleaned_rules[i]])
    else:
        indices.append(None)

In [33]:
import pickle5

ModuleNotFoundError: No module named 'pickle5'

In [27]:
# import pickle
# a = None
# with open('../../data/filled_html/filled_html_sathvik/1389603402631.json_tagged.pkl','rb') as f:
#     a = pickle.load(f)

In [37]:
def template_tag_pipeline(filename):
    import pickle
    ale = {}
    with open(filename,'rb') as f:
        ut_entities = pickle.load(f)
        out_entities = []
        
        rules = list(ut_entities.keys())
        for ele in rules:
            mini_entities = []
            for jele in ut_entities[ele]:
                s,e,i,x = jele
                mini_entities.append({"start":s,"end":e,"label":i})
            out_entities.append(mini_entities)
        entities = {}
        ale = set()
        for i in range(len(out_entities)):
            mrule = rules[i]
            for ele in out_entities[i]:
            entities[str(i)] = {
                "cleanedUpEnts": [mrule[ele['start']:ele['end']] for ele in out_entities[i] if ele['label'] not in ['time','Place','Object','Penalty','Legal Doc']]
            }
    new_rules = []
    new_entities = {}
    new_indices = []
    counter = 0
    for i in range(len(rules)):
        val  = rules[i]
        v1 = re.match('^[0-9]+(substituted|inserted)',val.lower())
        v2 = re.match('^Explanation',val)
        v3 = re.match('^NOTE:',val)
        if (v1 is not None) or (v2 is not None) or (v3 is not None):
            pass
        else:
            new_rules.append(rules[i])
            new_entities[str(counter)] = entities[str(i)]
            counter+=1
    
    ex = template_tagging(new_rules,new_entities)
#     print('--------------------------------------------------------')
    return ex

In [44]:
import os
from glob import glob
from tqdm import tqdm as tqdm
for file in tqdm(glob('../../data/filled_html/filled_html_sathvik/*.pkl')):
    bn = os.path.basename(file)
    ex = template_tag_pipeline(file)
    with open('../../data/filled_html/filled_html_shravya/'+bn,'wb') as f:
        pickle.dump(ex,f)

 15%|█▌        | 3/20 [00:00<00:00, 17.69it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


 30%|███       | 6/20 [00:00<00:01, 11.88it/s]

34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


 40%|████      | 8/20 [00:00<00:01, 11.06it/s]

...


187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436

 80%|████████  | 16/20 [00:01<00:00,  5.66it/s]

84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


100%|██████████| 20/20 [00:02<00:00,  9.17it/s]

160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62

In [25]:
# import pickle
# with open('/home/buggi/RA/data/parse_dumps/TEMPLATING NER/1538990677517_tagged.pkl','rb') as f:
#     ut_entities = pickle.load(f)

In [24]:
# ut_entities